In [1]:
!pip -q  install scikeras nltk

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.7.0 which is incompatible.


In [2]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd 
##load dataset
data=gutenberg.raw('shakespeare-hamlet.txt')
#save to file
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [3]:
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


#load the dataset

with open('hamlet.txt','r') as file:
    text=file.read().lower()

##tokenize the text
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1

2025-01-04 13:29:14.021165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-04 13:29:14.021236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-04 13:29:14.022761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-04 13:29:14.033293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-04 13:29:15.528173: W tensorflow/compiler/tf2

In [4]:
total_words

4818

In [5]:
#create input sequences
inputsequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
            n_gram_sequence=token_list[:i+1]
            inputsequences.append(n_gram_sequence)
            
                   

In [6]:
inputsequences[0:5]

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886]]

In [7]:
#pad sequences

max_seq_len=max([len(x) for x in inputsequences])

In [8]:
max_seq_len

14

In [9]:
input_sequences=np.array(pad_sequences(inputsequences, maxlen=max_seq_len,padding='pre'))

In [10]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [11]:
import tensorflow as tf 
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [12]:
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [13]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [14]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [16]:
#y

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [27]:
cb=tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    restore_best_weights=True
)

In [35]:
#train LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

#define the model 
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_seq_len))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(200, return_sequences=False))
model.add(Dense(total_words,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
##train
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1,
                 callbacks=[cb])

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.0317 - loss: 7.1230 - val_accuracy: 0.0326 - val_loss: 6.7318
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.0391 - loss: 6.4210 - val_accuracy: 0.0392 - val_loss: 6.8096
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0441 - loss: 6.3100 - val_accuracy: 0.0468 - val_loss: 6.8508
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.0562 - loss: 6.0858 - val_accuracy: 0.0476 - val_loss: 6.8793
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0554 - loss: 5.9706 - val_accuracy: 0.0521 - val_loss: 6.9143
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0637 - loss: 5.8341 - val_accuracy: 0.0552 - val_loss: 6.9768
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0743 - loss: 5.6458 - val_accuracy: 0.0618 - val_loss: 7.0587
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0759 - loss: 5.4920 - v

In [40]:
def predict_next_word(model,tokenizer, text, max_seq_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_seq_len:
        token_list=token_list[-(max_seq_len-1):]
    token_list=pad_sequences([token_list],maxlen=max_seq_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word, index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [41]:
input_text="To be or not to be"
print(f"Input Text: {input_text}")
max_seq_len=model.input_shape[1]+1 #14
next_word=predict_next_word(model, tokenizer,input_text,max_seq_len)
print(f"Next word prediction: {next_word}")

Input Text: To be or not to be
Next word prediction: not


In [42]:
model.save('next_word_lstm.h5')
#save tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)